In [6]:
import pandas as pd
from datetime import date
import numpy as np
from dateutil.relativedelta import relativedelta

In [14]:
# Load main subscription data
df_main = pd.read_excel('/Users/emirkabasoglu/Desktop/Eloomi/VS Codes/Jupyter Notebook Scripts/Account Allignment/2023-10-30-T-datasheets-subscriptions.xlsx')
# df_main = pd.read_excel('/Users/jesperfriislarnaes/Library/CloudStorage/OneDrive-eloomi/Scripts/New_Dashboard/test-data-a.xlsx')

In [15]:
# Set dates - report date is the month after the one being reported on
report_date = date(2022,1,1)
ltm_period_start = report_date + relativedelta(months=-48)

In [16]:
# FUNCTION TO CALCULATE ARR
def arr_ltm_calculations(df, name):
    # Function to get ARR main grid
    def ltm_arr(df, ltm_period_start, ltm_period_end):
        while ltm_period_start < ltm_period_end:
            revenue_period = ltm_period_start + relativedelta(months=+1, days=-1)
            mask_active = (df['start'].dt.date <= revenue_period) & (df['end'].dt.date >= revenue_period)
            if df.empty:
                df[f'{revenue_period}'] = np.nan
            else:
                df[f'{revenue_period}'] = df.loc[mask_active]['value']
            ltm_period_start = ltm_period_start + relativedelta(months=+1)
        return df

    # Run function using dataframe as input and current_end_of_month to define the ltm period range
    df_ltm_arr = ltm_arr(df, ltm_period_start, report_date)

    # Group by ids to get grid on account level
    revenue_period_cols = list(df_ltm_arr.iloc[:,12:])
    df_ltm_arr_agg = df_ltm_arr.groupby('id')[revenue_period_cols].sum()
    df_ltm_arr_agg.to_excel(f'{name}-ltm-arr-per-account.xlsx')

    # Set up dataframe for arr ending
    df_arr_ending = df_ltm_arr_agg.agg(['sum']).rename(index={'sum': f'{name}_arr_ending'})

    # Set up dataframe for arr churn 
    df_arr_churn = df_ltm_arr_agg.diff(axis=1)[df_ltm_arr_agg.eq(0)].agg(['sum']).rename(index={'sum': f'{name}_arr_churn'})

    # Set up dataframe for new logo arr
    df_arr_new_logo = df_ltm_arr_agg.diff(axis=1)[df_ltm_arr_agg.eq(0).shift(axis=1)].agg(['sum']).rename(index={'sum': f'{name}_arr_new_logo'})

    # Set up dataframe for upsell arr
    mask_arr_upsell = df_ltm_arr_agg[df_ltm_arr_agg.gt(0)].diff(axis=1).fillna(0).gt(0)
    df_arr_upsell = df_ltm_arr_agg.diff(axis=1)[mask_arr_upsell].agg(['sum']).rename(index={'sum': f'{name}_arr_upsell'})

    # Set up dataframe for downsell arr
    mask_arr_downsell = df_ltm_arr_agg[df_ltm_arr_agg.gt(0)].diff(axis=1).fillna(0).lt(0)
    df_arr_downsell = df_ltm_arr_agg.diff(axis=1)[mask_arr_downsell].agg(['sum']).rename(index={'sum': f'{name}_arr_downsell'})

    # Set up dataframe for arr ending logo count
    df_arr_ending_logo_count = df_ltm_arr_agg.replace(0,np.nan).count().to_frame(name=f'{name}_arr_ending_logo_count').transpose()

    # Set up dataframe for arr churn logo count
    df_arr_churn_logo_count = df_ltm_arr_agg.diff(axis=1)[df_ltm_arr_agg.eq(0)].replace(0,np.nan).count().to_frame(name=f'{name}_arr_churn_logo_count').transpose()

    # Set up dataframe for arr new logo count
    df_arr_new_logo_count = df_ltm_arr_agg.diff(axis=1)[df_ltm_arr_agg.eq(0).shift(axis=1)].replace(0,np.nan).count().to_frame(name=f'{name}_arr_new_logo_count').transpose()

    # Set up dataframe for arr upsell logo count
    mask_arr_upsell = df_ltm_arr_agg[df_ltm_arr_agg.gt(0)].diff(axis=1).fillna(0).gt(0)
    df_arr_upsell_logo_count = df_ltm_arr_agg.diff(axis=1)[mask_arr_upsell].replace(0,np.nan).count().to_frame(name=f'{name}_arr_upsell_logo_count').transpose()

    # Set up dataframe for downsell arr logo count
    mask_arr_downsell = df_ltm_arr_agg[df_ltm_arr_agg.gt(0)].diff(axis=1).fillna(0).lt(0)
    df_arr_downsell_logo_count = df_ltm_arr_agg.diff(axis=1)[mask_arr_downsell].replace(0,np.nan).count().to_frame(name=f'{name}_arr_downsell_logo_count').transpose()

    # Set up concatenated dataframe
    arr_dfs = pd.concat([df_arr_new_logo, df_arr_churn, df_arr_upsell, df_arr_downsell, df_arr_ending, df_arr_new_logo_count, df_arr_churn_logo_count, df_arr_upsell_logo_count, df_arr_downsell_logo_count, df_arr_ending_logo_count], axis=0)
    return arr_dfs

In [17]:
# ALL GEOS
# Size split total arr
# total arr
df_total_filter = df_main
df_total = arr_ltm_calculations(df_total_filter, 'all_geos_total')

# # smb total arr
# df_smb_total_filter = df_main[df_main['size'] == 'smb']
# df_smb_total = arr_ltm_calculations(df_smb_total_filter, 'all_geos_smb')

# # mm total arr
# df_mm_total_filter = df_main[df_main['size'] == 'mm']
# df_mm_total = arr_ltm_calculations(df_mm_total_filter, 'all_geos_mm')

# # ent total arr
# df_ent_total_filter = df_main[df_main['size'] == 'ent']
# df_ent_total = arr_ltm_calculations(df_ent_total_filter, 'all_geos_ent')

# # Product split total arr
# # Software total
# df_software_total_filter = df_main[df_main['product'] == 'software']
# df_software_total = arr_ltm_calculations(df_software_total_filter, 'all_geos_software')

# # Content total
# df_content_total_filter = df_main[df_main['product'] == 'content']
# df_content_total = arr_ltm_calculations(df_content_total_filter, 'all_geos_content')

# # Type split total arr
# # People total
# df_people_total_filter = df_main[df_main['type'] == 'people']
# df_people_total = arr_ltm_calculations(df_people_total_filter, 'all_geos_people')

# # Infinite total
# df_infinite_total_filter = df_main[df_main['type'] == 'infinite']
# df_infinite_total = arr_ltm_calculations(df_infinite_total_filter, 'all_geos_infinite')

# Create output excel file for geo 
# # create a excel writer object
with pd.ExcelWriter(f"db-{report_date}-all-geos-arr.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_total.to_excel(writer, sheet_name="all-geos-total-arr")
    # df_smb_total.to_excel(writer, sheet_name="all-geos-smb-arr")
    # df_mm_total.to_excel(writer, sheet_name="all-geos-mm-arr")
    # df_ent_total.to_excel(writer, sheet_name="all-geos-ent-arr")
    # df_software_total.to_excel(writer, sheet_name="all-geos-software-arr")
    # df_content_total.to_excel(writer, sheet_name="all-geos-content-arr")
    # df_people_total.to_excel(writer, sheet_name="all-geos-people-arr")
    # df_infinite_total.to_excel(writer, sheet_name="all-geos-infinite-arr")

In [ ]:
# ---=== END ===---